In [1]:
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys
from sklearn import datasets
import matplotlib.pyplot as plt

df = pd.read_csv('historical_data1_Q12005_clean.csv')
df2 = pd.read_csv('historical_data1_Q22005_clean.csv')

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# NOTE: DOES NOT USE mortgage_insurance_perc
dummies = pd.get_dummies(df[['first_time_homebuyer_flag','occupancy_status', 
                             'channel','prepayment_penalty_mortgage_flag',  'loan_purpose']])

y = df.orig_interest_rate
X_ = df.drop(['first_payment_date', 'first_time_homebuyer_flag', 'maturity_date',
              'metropolitan_stat_area', 
              'occupancy_status', 'channel', 'prepayment_penalty_mortgage_flag', 'product_type',
             'property_state', 'property_type', 'postal_code', 'loan_sequence_no',
             'loan_purpose', 'orig_loan_term', 'seller_name', 'service_name',
              'orig_interest_rate','mortgage_insurance_perc'], axis=1).astype('float64')
X = pd.concat([X_, dummies[['first_time_homebuyer_flag_Y', 'occupancy_status_I',
                            'occupancy_status_O', 'occupancy_status_S',
                           'channel_B', 'channel_C', 'channel_R', 'channel_T',
                           'prepayment_penalty_mortgage_flag_Y', 'loan_purpose_C',
                           'loan_purpose_N', 'loan_purpose_P']]], axis=1)

In [3]:
# NOTE: DOES NOT USE mortgage_insurance_perc
dummies2 = pd.get_dummies(df2[['first_time_homebuyer_flag','occupancy_status', 
                             'channel','prepayment_penalty_mortgage_flag',  'loan_purpose']])

y_val = df2.orig_interest_rate
X_val_ = df2.drop(['first_payment_date', 'first_time_homebuyer_flag', 'maturity_date',
              'metropolitan_stat_area', 
              'occupancy_status', 'channel', 'prepayment_penalty_mortgage_flag', 'product_type',
             'property_state', 'property_type', 'postal_code', 'loan_sequence_no',
             'loan_purpose', 'orig_loan_term', 'seller_name', 'service_name',
              'orig_interest_rate','mortgage_insurance_perc'], axis=1).astype('float64')
X_val = pd.concat([X_val_, dummies2[['first_time_homebuyer_flag_Y', 'occupancy_status_I',
                            'occupancy_status_O', 'occupancy_status_S',
                           'channel_B', 'channel_C', 'channel_R', 'channel_T',
                           'prepayment_penalty_mortgage_flag_Y', 'loan_purpose_C',
                           'loan_purpose_N', 'loan_purpose_P']]], axis=1)

In [4]:
def processSubset(feature_set):
# Fit model on feature_set and calculate RSS
    model = MLPRegressor()
    model.fit(X[list(feature_set)], y)
    #MAE, RMS, MAPE
    RMS_train = mean_squared_error(y, model.predict(X[list(feature_set)]))
    RMS_test = mean_squared_error(y_val, model.predict(X_val[list(feature_set)]))
    
    MAE_train = mean_absolute_error(y, model.predict(X[list(feature_set)]))
    MAE_test = mean_absolute_error(y_val, model.predict(X_val[list(feature_set)]))
    
    MAPE_train = np.mean(np.abs((y - model.predict(X[list(feature_set)])) / y)) * 100
    MAPE_test = np.mean(np.abs((y_val - model.predict(X_val[list(feature_set)])) / y_val)) * 100
    
    
    RSS = ((model.predict(X_val[list(feature_set)]) - y_val) ** 2).sum()
    return {"model":model, "RSS":RSS, "feature":X[list(feature_set)], 
            "RMS_train": RMS_train, "RMS_test": RMS_test,
            "MAE_train": MAE_train, "MAE_test": MAE_test,
            "MAPE_train": MAPE_train, "MAPE_test": MAPE_test}

In [5]:
def forward(predictors):
    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
        
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    # Choose the model with the highest RSS
    best_model = models.loc[models['RMS_test'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    # Return the best model, along with some other useful information about the model
    return best_model

models_forward = pd.DataFrame(columns=["RSS", "model", "feature", "RMS_train", 
                                "RMS_test", "MAE_train", "MAE_test", "MAPE_train", "MAPE_test"])
tic = time.time()
predictors = []
for i in range(1,len(X.columns)+1):
    models_forward.loc[i] = forward(predictors)
    predictors = list(models_forward.loc[i]["feature"])
    
toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

('Processed ', 19, 'models on', 1, 'predictors in', 120.98584699630737, 'seconds.')
('Processed ', 18, 'models on', 2, 'predictors in', 189.02842903137207, 'seconds.')
('Processed ', 17, 'models on', 3, 'predictors in', 177.61807298660278, 'seconds.')
('Processed ', 16, 'models on', 4, 'predictors in', 162.59874391555786, 'seconds.')
('Processed ', 15, 'models on', 5, 'predictors in', 140.66894698143005, 'seconds.')
('Processed ', 14, 'models on', 6, 'predictors in', 224.45255303382874, 'seconds.')
('Processed ', 13, 'models on', 7, 'predictors in', 262.61804604530334, 'seconds.')
('Processed ', 12, 'models on', 8, 'predictors in', 406.18085193634033, 'seconds.')
('Processed ', 11, 'models on', 9, 'predictors in', 375.66637086868286, 'seconds.')
('Processed ', 10, 'models on', 10, 'predictors in', 160.7488090991974, 'seconds.')
('Processed ', 9, 'models on', 11, 'predictors in', 255.56582808494568, 'seconds.')
('Processed ', 8, 'models on', 12, 'predictors in', 152.56870579719543, 'sec

In [6]:
models_forward

,RSS,model,feature,RMS_train,RMS_test,MAE_train,MAE_test,MAPE_train,MAPE_test
1,52357.044592,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O 0 ...,0.134554,0.129066,0.280965,0.277595,5.047404,4.757592
2,50350.317759,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue 0...,0.124875,0.124119,0.270000,0.271501,4.845406,4.653959
3,47700.437849,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.135230,0.117587,0.282076,0.263109,5.112382,4.560173
4,47502.150762,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.124226,0.117098,0.269163,0.263609,4.854035,4.537930
5,47689.220038,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.125371,0.117559,0.270166,0.263550,4.874348,4.539379
6,44960.979164,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.136044,0.110834,0.282112,0.254677,5.137950,4.432770
7,49223.022846,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.118394,0.121340,0.263687,0.270299,4.727095,4.626312
8,47311.957733,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.153747,0.116629,0.304331,0.261961,5.560981,4.595403
9,46363.358138,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.123037,0.114291,0.268519,0.260815,4.847250,4.494780
10,48822.457724,"MLPRegressor(activation='relu', alpha=0.0001, ...",occupancy_status_O orig_loantovalue ...,0.165618,0.120353,0.318270,0.266371,5.826650,4.692764


In [7]:
def getBestModel(models):
    length = len(models.index)
    bestModel = models.loc[1]
    RMS = models.loc[1]["RMS_test"]
    for i in range (1, length + 1):
        if models.loc[i]["RMS_test"] < RMS:
            bestModel = models.loc[i]
            RMS = models.loc[i]["RMS_test"]
    return bestModel

In [8]:
bestmodel_forward = getBestModel(models_forward)
print (bestmodel_forward["RMS_test"])

0.108348075418
